In [1]:
# standard python imports
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np
import copy

# pytorch imports
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR

In [2]:
data = np.load('exp_io_data\CoupledPendula_mean_in7_out7_Tmax2.0_data.npz')
for key, val in data.items():
    print(key)
    exec(key +'=val')

xlist
exp_out_list


In [3]:
x=xlist
y=exp_out_list
#print(y)

In [4]:
print(x.shape)
print(y.shape)

(2000, 14)
(2, 60, 2000, 7, 2)


In [5]:
#y1=y[0:1,29:30,:,:,0:1]
print(y.shape)

#(Nrepeat, Ntimesteps, Nx, output_dim, 2)  <-- 2 = (final position, final velocity)
#y1_np=torch.from_numpy(y1)
#y1=torch.transpose(y1_np,1,2)
#print(y1.shape)
y2=y[0:1,:,0:1000,:,0:1]
#(Nrepeat, Ntimesteps, Nx, output_dim, 2)  <-- 2 = (final position, final velocity)
y2_np=torch.from_numpy(y2)
y2=torch.transpose(y2_np,1,2)
print(y2.shape)
#print(y1.shape)

(2, 60, 2000, 7, 2)
torch.Size([1, 1000, 60, 7, 1])


In [6]:
y3=y2.squeeze(0).squeeze(3)
print(y3.shape)

torch.Size([1000, 60, 7])


In [12]:
y_train=y3[:,3:4,:]
print(y_train.shape)

torch.Size([1000, 1, 7])


In [8]:
x_train=x[0:1000,:]
print(x_train.shape)
print(x)

(1000, 14)
[[0.9515082  0.02859658 0.36347437 ... 0.35507327 0.8292224  0.4047569 ]
 [0.25135255 0.7703726  0.775273   ... 0.14982301 0.15127295 0.7362834 ]
 [0.19514024 0.16856277 0.07611024 ... 0.7874241  0.22519547 0.09913057]
 ...
 [0.47233897 0.5813042  0.5970881  ... 0.4770273  0.8321323  0.0551641 ]
 [0.9792367  0.9304237  0.7407218  ... 0.07495439 0.42210817 0.49420857]
 [0.77542037 0.6565631  0.83288056 ... 0.03630865 0.04533863 0.08364367]]


In [19]:



class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()

        self.layer1 = nn.Linear(14, 14 ,bias=True)
        self.layer2 = nn.Linear(14, 14,bias=True)
        self.layer3 = nn.Linear(14, 7,bias=True)


    def forward(self, x):
        # Max pooling over a (2, 2) window
        
        x = torch.flatten(x, 1) # flatten all dimensions except the batch dimension
        x = x+0.1*self.layer1(x)
        x = x+0.1*self.layer2(x)
        x = torch.sigmoid(self.layer3(x))
        #x = F.relu(self.layer4(x))
        #x = F.relu(self.layer5(x))
        #x = torch.sigmoid(self.layer6(x))
        return x


net = Net()
print(net)

Net(
  (layer1): Linear(in_features=14, out_features=14, bias=True)
  (layer2): Linear(in_features=14, out_features=14, bias=True)
  (layer3): Linear(in_features=14, out_features=7, bias=True)
)


In [20]:
params = list(net.parameters())
print(len(params))
print(params[0].size())

6
torch.Size([14, 14])


In [21]:
x_data=torch.tensor(x_train)
input = x_data
out = net(input)
print(out)
print(out.shape)

tensor([[0.4259, 0.6515, 0.5182,  ..., 0.5584, 0.5824, 0.4887],
        [0.5059, 0.6381, 0.5155,  ..., 0.5571, 0.5858, 0.5471],
        [0.5772, 0.6640, 0.4657,  ..., 0.5410, 0.4751, 0.4539],
        ...,
        [0.5624, 0.6671, 0.5327,  ..., 0.6123, 0.5856, 0.5003],
        [0.5302, 0.6058, 0.5137,  ..., 0.5618, 0.5126, 0.5298],
        [0.6179, 0.6268, 0.5602,  ..., 0.5695, 0.4377, 0.4946]],
       grad_fn=<SigmoidBackward>)
torch.Size([1000, 7])


In [22]:
net.zero_grad()
out.backward(torch.randn(1000,7))

In [24]:
target=y_train
target_new=target.view(1000,7)
print(target_new.shape)
criterion=nn.MSELoss()
loss=criterion(out,target_new)
print(loss)

torch.Size([1000, 7])
tensor(0.8105, grad_fn=<MseLossBackward>)


In [26]:
import torch.optim as optim
optimizer = torch.optim.Adam(params, lr=0.01, betas=(0.9, 0.999), eps=1e-09, weight_decay=0.1, amsgrad=False)  #adam optimizer
    
for i in range(200):
    optimizer.zero_grad()   # zero the gradient buffers
    output = net(input)
    loss = criterion(output, target_new)
    loss.backward()
    optimizer.step()
    print(loss)


tensor(0.5865, grad_fn=<MseLossBackward>)
tensor(0.5851, grad_fn=<MseLossBackward>)
tensor(0.5862, grad_fn=<MseLossBackward>)
tensor(0.5872, grad_fn=<MseLossBackward>)
tensor(0.5876, grad_fn=<MseLossBackward>)
tensor(0.5874, grad_fn=<MseLossBackward>)
tensor(0.5870, grad_fn=<MseLossBackward>)
tensor(0.5865, grad_fn=<MseLossBackward>)
tensor(0.5860, grad_fn=<MseLossBackward>)
tensor(0.5858, grad_fn=<MseLossBackward>)
tensor(0.5858, grad_fn=<MseLossBackward>)
tensor(0.5860, grad_fn=<MseLossBackward>)
tensor(0.5863, grad_fn=<MseLossBackward>)
tensor(0.5867, grad_fn=<MseLossBackward>)
tensor(0.5869, grad_fn=<MseLossBackward>)
tensor(0.5871, grad_fn=<MseLossBackward>)
tensor(0.5870, grad_fn=<MseLossBackward>)
tensor(0.5869, grad_fn=<MseLossBackward>)
tensor(0.5866, grad_fn=<MseLossBackward>)
tensor(0.5864, grad_fn=<MseLossBackward>)
tensor(0.5862, grad_fn=<MseLossBackward>)
tensor(0.5861, grad_fn=<MseLossBackward>)
tensor(0.5862, grad_fn=<MseLossBackward>)
tensor(0.5863, grad_fn=<MseLossBac

In [27]:
x_test=x[1010:1020,:]
print(x_test.shape)
y1=y[0:1,:,1010:1020,:,0:1]
#(Nrepeat, Ntimesteps, Nx, output_dim, 2)  <-- 2 = (final position, final velocity)
y1_np=torch.from_numpy(y1)
y1=torch.transpose(y1_np,1,2)

(10, 14)


In [28]:
y4=y1.squeeze(0).squeeze(3)
print(y4.shape)

y_test=y4[:,29:30,3:4]
print(y_test.shape)

torch.Size([10, 60, 7])
torch.Size([10, 1, 1])


In [29]:
x_test=torch.tensor(x_test)
test_out=net(x_test)
#LOSS = criterion(test_out, y_test)
print(test_out)
print(y_test)

tensor([[0.3317, 0.3297, 0.3363, 0.3311, 0.3287, 0.3300, 0.3321],
        [0.3459, 0.3321, 0.3501, 0.3301, 0.3271, 0.3614, 0.3401],
        [0.2821, 0.2818, 0.2937, 0.2800, 0.2697, 0.2847, 0.2920],
        [0.3477, 0.3325, 0.3445, 0.3321, 0.3245, 0.3381, 0.3588],
        [0.3653, 0.3586, 0.3638, 0.3558, 0.3573, 0.3843, 0.3662],
        [0.2942, 0.2795, 0.3090, 0.2992, 0.2730, 0.3037, 0.3077],
        [0.2964, 0.2910, 0.3216, 0.3077, 0.3009, 0.3221, 0.3171],
        [0.2981, 0.3097, 0.3130, 0.2947, 0.2960, 0.3070, 0.3144],
        [0.3355, 0.3374, 0.3487, 0.3207, 0.3199, 0.3288, 0.3325],
        [0.2995, 0.2949, 0.2904, 0.2861, 0.2849, 0.2890, 0.2957]],
       grad_fn=<SigmoidBackward>)
tensor([[[-0.0254]],

        [[-0.5383]],

        [[-0.1750]],

        [[-1.9699]],

        [[-0.8048]],

        [[-1.3674]],

        [[-0.3631]],

        [[-0.1402]],

        [[ 0.1298]],

        [[ 0.5736]]])


In [ ]:

class Net2(nn.Module):

    def __init__(self):
        super(Net2, self).__init__()

        self.layer1 = nn.Linear(14, 120)
        self.layer2 = nn.Linear(120, 120)
        self.layer3 = nn.Linear(120, 120)
        self.layer4 = nn.Linear(120, 84)
        self.layer5 = nn.Linear(84, 42)
        self.layer6 = nn.Linear(42, 7)

    def forward(self, x):
        # Max pooling over a (2, 2) window
        
        x = torch.flatten(x, 1) # flatten all dimensions except the batch dimension
        x = F.relu(self.layer1(x))
        x = x+F.relu(self.layer2(x))
        x = x+F.relu(self.layer3(x))
        x = F.relu(self.layer4(x))
        x = F.tanh(self.layer5(x))
        x = F.relu(self.layer6(x))
        return x


net2 = Net2()
print(net2)

params2 = list(net2.parameters())
print(len(params2))
print(params2[0].size())

In [ ]:
x_data=torch.tensor(x)
input = x_data
out = net2(input)
print(out)
print(out.shape)

In [ ]:
target=y1[0:1000,:,:]
target_new=target.view(1000,7)

In [ ]:
import torch.optim as optim
optimizer = torch.optim.Adam(params, lr=0.001, betas=(0.9, 0.999), eps=1e-06, weight_decay=0.1, amsgrad=False)  #adam optimizer
    
for i in range(200):
    optimizer.zero_grad()   # zero the gradient buffers
    output = net2(input)
    loss = criterion(output, target_new)
    loss.backward()
    optimizer.step()
    print(loss)
